<a href="https://colab.research.google.com/github/Rana-ob/learn--for-GP/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Pillow


In [1]:
!pip install opencv-python

In [4]:
!apt install tesseract-ocr -y
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
import cv2
from PIL import Image
import pytesseract

In [7]:
from google.colab import files
uploaded = files.upload()

Saving Screenshot 2024-02-17 131614.png to Screenshot 2024-02-17 131614.png


In [12]:
# Replace with your uploaded filename
im_file = "Screenshot 2024-02-17 131614.png"  # Use the actual uploaded filename
im = Image.open(im_file)

# Print info
print(im)
im.show()

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=994x756 at 0x7FCBAD966910>
